In [2]:
import carla, time, pygame, math, random, cv2
from carla import Sensor
import math
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    spectator_transform = carla.Transform(back_location, transform.rotation)
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.mercedes.coupe_2020', rotation=None):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    if rotation:
        spawn_point.rotation = rotation
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle


def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x = 2.0, z=1.0)), horizontal_fov=10, range=100, vertical_fov=10, points_per_second=1500, sensor_tick=0.0):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('range', str(range))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar_bp.set_attribute('points_per_second', str(points_per_second))
    radar_bp.set_attribute('sensor_tick', str(sensor_tick))
    radar = world.spawn_actor(radar_bp, transform=transform, attach_to=attach_to)
    return radar

def remove_all(world):
    for a in world.get_actors().filter('vehicle.*'):
        a.destroy()
    for a in world.get_actors().filter('sensor.*'):
        a.destroy()


pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client.load_world('Town03')

RuntimeError: time-out of 10000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [1]:
from utils.detector import Detector
from utils.interpolator import Interpolator
from utils.imagewarp import ImageWarp

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

In [1]:
import carla, time, pygame, math, random, cv2
from carla import Sensor
import math
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    spectator_transform = carla.Transform(back_location, transform.rotation)
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.mercedes.coupe_2020', rotation=None):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    if rotation:
        spawn_point.rotation = rotation
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle


def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), fov=90.0, width=800, height=600, sensor_tick=0.0):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera_bp.set_attribute('fov', str(fov))
    camera_bp.set_attribute('sensor_tick', str(sensor_tick))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x = 2.0, z=1.0)), horizontal_fov=10, range=100, vertical_fov=10, points_per_second=1500, sensor_tick=0.0):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('range', str(range))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar_bp.set_attribute('points_per_second', str(points_per_second))
    radar_bp.set_attribute('sensor_tick', str(sensor_tick))
    radar = world.spawn_actor(radar_bp, transform=transform, attach_to=attach_to)
    return radar

def remove_all(world):
    for a in world.get_actors().filter('vehicle.*'):
        a.destroy()
    for a in world.get_actors().filter('sensor.*'):
        a.destroy()

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
from multiprocessing import Pool, Queue

processed_output = Queue()
pool = Pool(processes=4)

import manual_control
import imageprocessing
from importlib import reload
reload(manual_control)
reload(imageprocessing)

remove_all(world)

game_loop = manual_control.setup()

vehicle = world.get_actors().filter('vehicle.*')[0]
front_camera = spawn_camera(attach_to=vehicle, transform=carla.Transform(
        carla.Location(x=0.3, y=0.0, z=1.5), # specchietto retrovisore -> x=0.3, y=0.0, z=1.5), logo = x=2.3, z=0.7
        carla.Rotation(pitch=-10.0)
    ),
    sensor_tick=0.1,
    width=800, height=600
)

video_output = np.zeros((240, 320, 4), dtype=np.uint8)

cv2.namedWindow('Processed', cv2.WINDOW_NORMAL)

def camera_callback(image):
    global video_output

    try:
        video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
        video_output = video_output[:, :, :3]

        # Crop the image (zoom)
        crop_width = 320
        crop_height = 240
        height_adjust = 50
        start_x = (video_output.shape[1] - crop_width) // 2
        start_y = (video_output.shape[0] - crop_height) // 2 - height_adjust
        cropped_img = video_output[start_y:start_y + crop_height, start_x:start_x + crop_width]

        try:
            pool.apply_async(
                imageprocessing.process_image, args=(cropped_img,),
                callback=lambda result: (print('time: ', result[1]) if result[1] > 0.06 else print(), processed_output.put(result[0]))
            )
        except Exception as e:
            print(e)

    except Exception as e:
        print(e.with_traceback())


front_camera.listen(lambda image: camera_callback(image))

try:
    game_loop.start(processed_output)
except KeyboardInterrupt:
    print('\nCancelled by user. Bye!')
    pool.close()
    pool.join()
    cv2.destroyAllWindows()
finally:
    pool.close()
    pool.join()
    cv2.destroyAllWindows()


Pool not running


Traceback (most recent call last):
  File "c:\Users\emanu\miniconda3\envs\carla-env\lib\multiprocessing\queues.py", line 238, in _feed
    send_bytes(obj)
  File "c:\Users\emanu\miniconda3\envs\carla-env\lib\multiprocessing\connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "c:\Users\emanu\miniconda3\envs\carla-env\lib\multiprocessing\connection.py", line 290, in _send_bytes
    nwritten, err = ov.GetOverlappedResult(True)
BrokenPipeError: [WinError 109] Pipe terminata
Traceback (most recent call last):
  File "c:\Users\emanu\miniconda3\envs\carla-env\lib\multiprocessing\queues.py", line 238, in _feed
    send_bytes(obj)
  File "c:\Users\emanu\miniconda3\envs\carla-env\lib\multiprocessing\connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "c:\Users\emanu\miniconda3\envs\carla-env\lib\multiprocessing\connection.py", line 280, in _send_bytes
    ov, err = _winapi.WriteFile(self._handle, buf, overlapped=Tr


Welcome to CARLA manual control.

Use ARROWS or WASD keys for control.

    W            : throttle
    S            : brake
    A/D          : steer left/right
    Q            : toggle reverse
    Space        : hand-brake

    L            : toggle next light type
    Z/X          : toggle right/left blinker

    TAB          : change sensor position
    ` or N       : next sensor
    [1-9]        : change to sensor [1-9]
    C            : change weather (Shift+C reverse)

    F1           : toggle HUD
    ESC          : quit

time:  0.06949806213378906
time:  0.0709998607635498
time:  0.0650014877319336
time:  0.07250022888183594
time:  0.0690000057220459
time:  0.06850028038024902
time:  0.06849813461303711
time:  0.07499861717224121
time:  0.0644991397857666
time:  0.06549978256225586
time:  0.07050204277038574
time:  0.06849837303161621
time:  0.07549858093261719
time:  0.06149721145629883
time:  0.06649947166442871
time:  0.06099748611450195



















time:  0.062999